# Imports

In [1]:
#Importar librerías necesarias para desarrollar el análisis
import importlib
import warnings

import pandas as pd
import numpy as np
import sklearn 
from sklearn.metrics import mean_squared_error, auc, log_loss, roc_auc_score, accuracy_score, confusion_matrix, classification_report, f1_score, recall_score, precision_score, make_scorer
import random as python_random
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy.stats import ks_2samp
import scorecardpy as sc
#import openpyxl

from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, PowerTransformer
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.feature_selection import SelectKBest, chi2, RFE, mutual_info_classif
from sklearn.model_selection import train_test_split 
from sklearn.pipeline import make_pipeline, Pipeline 
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.svm import SVC

from sklearn.metrics import roc_curve, auc, log_loss, roc_auc_score, accuracy_score, confusion_matrix, classification_report, f1_score, recall_score, precision_score, make_scorer

from sklearn.model_selection import KFold, ShuffleSplit, LeaveOneOut, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV 

from xgboost import XGBClassifier
from xgboost import XGBRegressor
import xgboost

import sys
import notebook
import jupyterlab

import os
from pathlib import Path

/Users/cristian/Documents/GitHub/adeslas-interview/.venv/lib/python3.13/site-packages/scorecardpy/germancredit.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
from dotenv import load_dotenv, find_dotenv
# Carga el .env más cercano hacia arriba en el árbol
load_dotenv(find_dotenv())

# Raíz del proyecto = carpeta que contiene el .env
PROJECT_ROOT = Path(find_dotenv()).parent

# DATA_PATH puede ser relativo (./data) o absoluto; aquí lo resolvemos desde el root
DATA_PATH = (PROJECT_ROOT / os.environ.get("DATA_PATH", "data")).resolve()

In [3]:
import sys, platform
print(sys.executable)
print(platform.python_version())

/Users/cristian/Documents/GitHub/adeslas-interview/.venv/bin/python
3.13.7


In [4]:
pd.set_option("display.max_columns", None)   # muestra todas las columnas
pd.set_option("display.width", 0)            # usa el ancho de la celda de Jupyter
pd.set_option("display.max_colwidth", None)  # no truncar el contenido de celdas

# Env

In [5]:
PROJ_ROOT = Path.cwd().parent 
sys.path.insert(0, str(PROJ_ROOT / "src"))


# Verifica
print("Añadido a sys.path:", sys.path[0])

from utils import feature_selection as fs

Añadido a sys.path: /Users/cristian/Documents/GitHub/adeslas-interview/src


# Funciones

# Load data

In [6]:
DATA_PATH = Path(os.environ["DATA_PATH"])

In [7]:
file_tab = DATA_PATH / "df_muestra_vars_eng.csv"
df_muestra_vars_eng = pd.read_csv(file_tab)
print(df_muestra_vars_eng.shape)



file_tab = DATA_PATH / "X_train.csv"
X_train = pd.read_csv(file_tab)
print(X_train.shape)

file_tab = DATA_PATH / "X_val.csv"
X_val = pd.read_csv(file_tab)
print(X_val.shape)

file_tab = DATA_PATH / "X_test.csv"
X_test = pd.read_csv(file_tab)
print(X_test.shape)



file_tab = DATA_PATH / "y_train.csv"
y_train = pd.read_csv(file_tab)
print(len(y_train))


file_tab = DATA_PATH / "y_val.csv"
y_val = pd.read_csv(file_tab)
print(len(y_val))


file_tab = DATA_PATH / "y_test.csv"
y_test = pd.read_csv(file_tab)
print(len(y_test))


selected_vars = pd.read_csv(DATA_PATH / "selected_vars.csv")["feature"].astype(str).tolist()
print(len(selected_vars))

(47547, 420)
(28528, 418)
(9510, 418)
(9509, 418)
28528
9510
9509
20


# Model

## XGBoost con parametros estandar

In [8]:


# =========================
# Entrenamiento
# =========================
from utils.model_training import (
    train_xgb_binary,
    evaluate_model,
    save_artifacts,
    default_xgb_params,
)

# (Opcional) ajustar algunos hiperparámetros base
params = default_xgb_params() | dict(
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
)

results = train_xgb_binary(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    selected_vars=selected_vars,
    xgb_params=params,
    early_stopping_rounds=100,
)

print("Best iteration:", results.best_iteration)
print("n_estimators efectivos:", results.best_n_estimators)
display(results.metrics)



Best iteration: 2
n_estimators efectivos: 3


,auc,ks,prauc,n
split,,,,
train,0.816360,0.484141,0.160692,28528
val,0.674062,0.268752,0.093998,9510


In [ ]:

# =========================
# Evaluación en Test
# =========================
from utils.model_training import to_1d_series

X_te = X_test[[c for c in selected_vars if c in X_test.columns]].copy()
y_te = to_1d_series(y_test)

eval_test = evaluate_model(results.model, splits={"test": (X_te, y_te)})
print("\nMétricas test:")
display(eval_test)



In [ ]:
# =========================
# Guardado de artefactos
# =========================
#ARTIF_PATH = DATA_PATH / "models"
#model_path, feats_path = save_artifacts(results, ARTIF_PATH)
#print(f"Modelo guardado en: {model_path}")
#print(f"Features usadas guardadas en: {feats_path}")
